In [20]:
import boto3
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.metrics import classification_report
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression
from precision_recall_cutoff import precision_recall_cutoff
from sklearn.feature_selection import RFE, RFECV
from sklearn.svm import SVR

s3= boto3.resource('s3')
bucket_name= 'morgan-gant-data448-bucket'
bucket= s3.Bucket(bucket_name)

file_key= 'turnover.csv'

bucket_object= bucket.Object(file_key)
file_object = bucket_object.get()
file_content_stream = file_object.get('Body')

#reading the datefile
turnover = pd.read_csv(file_content_stream)
turnover.head()

,satisfaction_level,last_evaluation,number_project,average_montly_hours,time_spend_company,Work_accident,left,promotion_last_5years,sales,salary
0,0.38,0.53,2,157,3,0,1,0,sales,low
1,0.80,0.86,5,262,6,0,1,0,sales,medium
2,0.11,0.88,7,272,4,0,1,0,sales,medium
3,0.72,0.87,5,223,5,0,1,0,sales,low
4,0.37,0.52,2,159,3,0,1,0,sales,low


In [14]:
#changing to dummy variables
turnover= pd.concat([turnover.drop(columns=['sales', 'salary'], axis=1), pd.get_dummies(turnover[['sales', 'salary']])], axis=1)

#engineering features from the decisoon tree model
turnover['interaction_1']=np.where(((turnover['satisfaction_level'] >= .115) &
                                   (turnover['satisfaction_level'] >= .465) &
                                   (turnover['number_project'] > 2.5)), 1, 0)

turnover['interaction_2']=np.where(((turnover['satisfaction_level'] >= .465) &
                                   (turnover['number_project'] <=2.5) &
                                   (turnover['last_evaluation'] <= .575)), 1, 0)
        
turnover['interaction3']=np.where(((turnover['satisfaction_level'] >= .465) &
                                    (turnover['time_spend_company'] <=4.5) &
                                    (turnover['average_montly_hours'] <=290.5)), 1,0)


In [15]:
#Defining input and target variables
x= turnover[['satisfaction_level', 'number_project', 'time_spend_company', 'interaction_1', 'interaction3']]
y= turnover['left']

#splitting the data
x_train, x_test, y_train, y_test= train_test_split(x,y, test_size= .2, stratify= y)

In [16]:
#Scaling the data
scaler=MinMaxScaler()
x_train= pd.DataFrame(scaler.fit_transform(x_train), columns= x_train.columns)
x_test= pd.DataFrame(scaler.fit_transform(x_test), columns= x_test.columns)

Random Forest

In [17]:
rf_param_grid = {'n_estimators': [100, 300, 500],
'min_samples_split': [10, 15],
'min_samples_leaf': [5, 7],
'max_depth' : [3, 5, 7]}

#runnning gridsearchCV
rf_grid_search= GridSearchCV(estimator= RandomForestClassifier(), param_grid= rf_param_grid, cv= 5, scoring= 'f1').fit(x_train, y_train)
    
#Extracting best hyperparamter combo
rf_grid_search.best_params_

{'max_depth': 7,
 'min_samples_leaf': 5,
 'min_samples_split': 10,
 'n_estimators': 300}

In [19]:
#building a random forest with best parameters
rf= RandomForestClassifier(**rf_grid_search.best_params_).fit(x_train, y_train)

#predicting on the test
rf_pred= rf.predict_proba(x_test)[:,1]

#changing likelyhoods to labels
rf_label= precision_recall_cutoff(y_test, rf_pred)

print(classification_report(y_test,rf_label))

              precision    recall  f1-score   support

           0       0.97      0.98      0.98      2286
           1       0.94      0.91      0.93       714

    accuracy                           0.96      3000
   macro avg       0.96      0.95      0.95      3000
weighted avg       0.96      0.96      0.96      3000



In [9]:
from sklearn.svm import SVC

SVM

In [ ]:
svm_param_grid = {'kernel': ['rbf', 'poly', 'sigmoid'],
'C': [0.01, 0.1, 1, 10],
'gamma': [0.001, 0.01, 0.1, 1]}

#runnning gridsearchCV
svm_grid_search= GridSearchCV(estimator= SVC(probability= True), param_grid= svm_param_grid, cv= 5, scoring= 'f1').fit(x_train, y_train)
    
#Extracting best hyperparamter combo
svm_grid_search.best_params_

{'C': 10, 'gamma': 1, 'kernel': 'rbf'}

In [12]:
#building a random forest with best parameters
svc= SVC(**svm_grid_search.best_params_, probability=True).fit(x_train, y_train)

#predicting on the test
svc_pred= svc.predict_proba(x_test)[:,1]

#changing likelyhoods to labels
svc_label= precision_recall_cutoff(y_test, svc_pred)

print(classification_report(y_test, svc_label))

              precision    recall  f1-score   support

           0       0.97      0.98      0.97      2286
           1       0.92      0.91      0.92       714

    accuracy                           0.96      3000
   macro avg       0.95      0.94      0.95      3000
weighted avg       0.96      0.96      0.96      3000



In [ ]:
#Based on my models and data splits, I would need to run some iterations on the both models to see which one performs the best continuously. For both models, the accuracy's, recalls and percisions are very similar.